# SETUP

## Import Libraries

In [52]:
from pymongo import MongoClient
import pandas as pd

## Read MongoDB

In [53]:
client = MongoClient()
db = client["test"]

## Initialize Output DataFrame

In [54]:
output_df = pd.DataFrame()

# MATCHAPPS

## Matchapps Initalization

### Create Matchapps DataFrame

In [55]:
matchapps_collection = db["matchapps"]
matchapps_df = matchapps_collection.find()

### Flatten, reorganize, and clean Data

In [56]:
matchapps_df = pd.json_normalize(matchapps_df)

matchapps_column_order = [
    "_id",
    "metadata.matchNumber",
    "metadata.scouterName",
    "metadata.robotTeam",
    "metadata.robotPosition",
    "leftStartingZone",
    "climb",
    "trapNotes",
    "autoNotes.near",
    "autoNotes.mid",
    "autoNotes.far",
    "autoNotes.amp",
    "autoNotes.miss",
    "teleNotes.near",
    "teleNotes.mid",
    "teleNotes.far",
    "teleNotes.amp",
    "teleNotes.miss"   
]

matchapps_df = matchapps_df[matchapps_column_order]
matchapps_df.sort_values("metadata.matchNumber", inplace=True)
matchapps_df

,_id,metadata.matchNumber,metadata.scouterName,metadata.robotTeam,metadata.robotPosition,leftStartingZone,climb,trapNotes,autoNotes.near,autoNotes.mid,autoNotes.far,autoNotes.amp,autoNotes.miss,teleNotes.near,teleNotes.mid,teleNotes.far,teleNotes.amp,teleNotes.miss
0,65e40d2fbe204c571ff498fa,1,Jim,3353,red_1,True,center,1,1,2,3,0,1,1,3,1,1,3
1,65e40d2fbe204c571ff498fe,1,Jim,4870,red_2,True,center,0,3,2,3,0,2,8,7,2,1,8
2,65e40d2fbe204c571ff49902,1,Jim,2356,red_3,False,center,1,4,4,1,2,4,9,1,3,5,5
3,65e40d2fbe204c571ff49906,1,Jim,4350,blue_1,False,park,0,4,3,2,4,2,7,7,8,8,8
4,65e40d2fbe204c571ff4990a,1,Jim,433,blue_2,True,failed,0,0,2,3,4,0,8,9,3,7,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390,65e40d37be204c571ff4be52,399,Jim,2981,red_3,True,park,0,4,0,0,1,1,8,6,6,3,7
2392,65e40d37be204c571ff4be5a,399,Jim,9484,blue_2,False,center,1,1,3,0,1,4,9,3,3,5,3
2388,65e40d37be204c571ff4be4a,399,Jim,2806,red_1,True,failed,1,1,1,0,1,1,0,1,7,6,0
2389,65e40d37be204c571ff4be4e,399,Jim,687,red_2,False,failed,1,0,0,4,1,4,3,0,9,7,4


## Matchapps Function and Variable Initializations

### Check Alliance Harmonization Function Definition

In [57]:
def check_team_harmonization_in_alliance(t1, t2, t3):
    t1_status, t2_status, t3_status = False, False, False
    if t1 not in ["failed", "none", "park"]:
        if t1 == t2:
            t1_status = True
            t2_status = True
        if t1 == t3:
            t1_status = True
            t3_status = True
    elif t2 not in ["failed", "none", "park"]:
        if t2 == t3:
            t2_status = True
            t3_status = True
    return t1_status, t2_status, t3_status

### Complete List of all Teams in DF

In [58]:
team_list = matchapps_df["metadata.robotTeam"].unique().tolist()

### Complete Range of All Matches

In [59]:
match_range = matchapps_df["metadata.matchNumber"].max()

## Matchapps Data Analysis

### Auto Data Analysis Calculations

#### Auto Speaker Notes Average (near, mid, far)

In [60]:
output_df["Auto Speaker Notes Near Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.near"].mean()
output_df["Auto Speaker Notes Mid Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.mid"].mean()
output_df["Auto Speaker Notes Far Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.far"].mean()

output_df["Auto Speaker Notes Total Avg"] = None

for team in team_list:
    output_df.at[team, "Auto Speaker Notes Total Avg"] = sum([output_df.at[team, "Auto Speaker Notes Near Avg"], output_df.at[team, "Auto Speaker Notes Mid Avg"], output_df.at[team, "Auto Speaker Notes Far Avg"]])

#### Auto Amp Notes Average

In [61]:
output_df["Auto Notes Amp Avg"] = matchapps_df.groupby("metadata.robotTeam")["autoNotes.amp"].mean()

#### Auto Percent of Miss Notes

In [62]:
output_df["Auto Percent of Missed Notes"] = None

 # CHANGE FORMAT LATER
output_df["Auto Max Missed Notes"] = None
for team in team_list:
    missed_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.miss"].sum()
    total_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.near"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.mid"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.far"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.amp"].sum()
    + missed_notes
    output_df.at[team, "Auto Percent of Missed Notes"] = missed_notes / total_notes
    
     # CHANGE FORMAT LATER

    output_df.at[team, "Auto Max Missed Notes"] = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["autoNotes.amp"].max()

#### Auto All Notes Average

In [63]:
output_df["Auto Notes Avg"] = (output_df["Auto Notes Amp Avg"] +
                               output_df["Auto Speaker Notes Near Avg"] +
                               output_df["Auto Speaker Notes Mid Avg"] +
                               output_df["Auto Speaker Notes Far Avg"]) / 4

### Teleop Data Analysis Calculations

#### Teleop Speaker Notes Average (near, mid, far)

In [64]:
output_df["Tele Speaker Notes Near Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.near"].mean()
output_df["Tele Speaker Notes Mid Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.mid"].mean()
output_df["Tele Speaker Notes Far Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.far"].mean()

output_df["Tele Speaker Notes Total Avg"] = None

for team in team_list:
    output_df.at[team, "Tele Speaker Notes Total Avg"] = sum([output_df.at[team, "Tele Speaker Notes Near Avg"], output_df.at[team, "Tele Speaker Notes Mid Avg"], output_df.at[team, "Tele Speaker Notes Far Avg"]])

#### Teleop Amp Notes Average

In [65]:
output_df["Tele Notes Amp Avg"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.amp"].mean()

 # CHANGE FORMAT LATER
output_df["Tele Max Amp Notes"] = matchapps_df.groupby("metadata.robotTeam")["teleNotes.amp"].max()

#### Teleop All Notes Average

In [66]:
output_df["Tele Notes Avg"] = (output_df["Tele Notes Amp Avg"] +
                               output_df["Tele Speaker Notes Near Avg"] +
                               output_df["Tele Speaker Notes Mid Avg"] +
                               output_df["Tele Speaker Notes Far Avg"]) / 4

 # CHANGE FORMAT LATER
output_df["Tele Max Notes"] = None

#### Teleop Percent of Missed Notes

In [67]:
output_df["Tele Percent of Missed Notes"] = None

 # CHANGE FORMAT LATER
output_df["Tele Max Missed Notes"] = None
for team in team_list:
    missed_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.miss"].sum()
    total_notes = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.near"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.mid"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.far"].sum()
    + matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.amp"].sum()
    + missed_notes
    output_df.at[team, "Percent of Missed Notes"] = missed_notes / total_notes
    
     # CHANGE FORMAT LATER

    output_df.at[team, "Tele Max Missed Notes"] = matchapps_df[matchapps_df["metadata.robotTeam"] == team]["teleNotes.amp"].max()

### Other Data Analysis Calculations

#### Trap Notes

In [68]:
output_df["Trap Notes Avg"] = matchapps_df.groupby("metadata.robotTeam")["trapNotes"].mean()

#### Percent of Successful Climbs and Successful Climbs Total

In [69]:
output_df["Percent of Successful Climbs"] = None
output_df["Successful Climbs Total"] = None

for team in team_list:
    climb_list = matchapps_df.loc[matchapps_df["metadata.robotTeam"] == team, "climb"].tolist()
    total_climbs = 0
    successful_climbs = 0
    for climb_type in climb_list:
        if climb_type == "failed":
            total_climbs += 1
        elif climb_type == "amp" or climb_type == "source" or climb_type == "center":
            total_climbs += 1
            successful_climbs += 1
    if total_climbs == 0:
        output_df.at[team, "Percent of Successful Climbs"] = 0
        
    else:
        output_df.at[team, "Percent of Successful Climbs"] = successful_climbs / total_climbs
    
    output_df.at[team, "Successful Climbs Total"] = successful_climbs

#### Harmonized Climbs Total

#### Percent of Harmonized Climbs out of Successful Climbs

In [70]:
# output_df["Percent of Harmonized Climbs out of Successful Climbs"] = 0

# output_df.index

# for team in team_list:
#     output_df.at[team, "Percent of Harmonized Climbs out of Successful Climbs"] = output_df.at[team, "Harmonized Climbs Total"] / output_df.at[team, "Successful Climbs Total"]

#### Percent of Times Robot Left Starting Zone During Autos

In [71]:
output_df["Percent of Times Robot Left Starting Zone During Autos"] = matchapps_df.groupby("metadata.robotTeam")["leftStartingZone"].mean()

### Custom Metric Data Analysis Calculations

#### Endgame Custom Metric

In [72]:
# output_df["Endgame Custom Metric"] = ((output_df["Percent of Successful Climbs"] * 5) ** 2 + (output_df["Percent of Harmonized Climbs out of Successful Climbs"] * 5) ** 2 + (output_df["Trap Notes Avg"] * 5) ** 2) / 3
# output_df["Endgame Custom Metric"] = output_df["Endgame Custom Metric"].apply(lambda x: round(x, 1))

# SUPERAPPS

## Supperapps Initalization

### Create Superapps DataFrame

In [73]:
superapps_collection = db["superapps"]
superapps_df = superapps_collection.find()

### Flatten, reorganize, and clean Data

In [74]:
superapps_df = pd.json_normalize(superapps_df)
superapps_df.sort_values("metadata.matchNumber", inplace=True)
superapps_df

,_id,defense,defended,__v,metadata.scouterName,metadata.matchNumber,metadata.robotTeam,metadata.robotPosition,fouls.inBot,fouls.damageBot,fouls.tipEntangBot,fouls.pinBot,fouls.podiumFoul,fouls.zoneFoul,fouls.stageFoul,fouls.overExtChute,humanShooter.highNotes.amp,humanShooter.highNotes.source,humanShooter.highNotes.center
0,65e40d2fbe204c571ff498fc,noDef,True,0,Jim,1,3353,red_1,1,0,0,0,0,0,0,1,False,True,True
1,65e40d2fbe204c571ff49900,fullDef,True,0,Jim,1,4870,red_2,0,1,1,1,0,1,1,0,NaN,NaN,NaN
2,65e40d2fbe204c571ff49904,someDef,False,0,Jim,1,2356,red_3,0,1,1,0,0,1,1,0,NaN,NaN,NaN
3,65e40d2fbe204c571ff49908,noDef,True,0,Jim,1,4350,blue_1,1,1,1,0,0,0,1,1,NaN,NaN,NaN
4,65e40d2fbe204c571ff4990c,noDef,True,0,Jim,1,433,blue_2,0,0,0,1,1,1,1,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390,65e40d37be204c571ff4be54,fullDef,True,0,Jim,399,2981,red_3,1,1,0,0,1,0,1,0,False,True,False
2392,65e40d37be204c571ff4be5c,noDef,False,0,Jim,399,9484,blue_2,0,1,0,1,0,0,1,1,True,True,True
2388,65e40d37be204c571ff4be4c,noDef,False,0,Jim,399,2806,red_1,0,0,0,0,1,1,1,0,NaN,NaN,NaN
2389,65e40d37be204c571ff4be50,noDef,False,0,Jim,399,687,red_2,1,0,1,0,1,0,1,1,NaN,NaN,NaN


## Superapps Data Analysis

### Average Fouls per Match for Different Foul Types

In [75]:
output_df["Average In Bot Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.inBot"].mean()
output_df["Average Damage Bot Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.damageBot"].mean()
output_df["Average Tip Entangle Bot Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.tipEntangBot"].mean()
output_df["Average Pin Bot Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.pinBot"].mean()
output_df["Average Podium Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.podiumFoul"].mean()
output_df["Average Zone Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.zoneFoul"].mean()
output_df["Average Stage Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.stageFoul"].mean()
output_df["Average Over Exc Chute Fouls per Match"] = superapps_df.groupby("metadata.robotTeam")["fouls.overExtChute"].mean()

### Average Total Fouls per Match

In [76]:
output_df["Average Fouls per Match"] = None

for team in team_list:
    output_df.at[team, "Average Fouls per Match"] = sum([
        output_df.at[team, "Average In Bot Fouls per Match"]
        + output_df.at[team, "Average Damage Bot Fouls per Match"]
        + output_df.at[team, "Average Tip Entangle Bot Fouls per Match"]
        + output_df.at[team, "Average Pin Bot Fouls per Match"]
        + output_df.at[team, "Average Podium Fouls per Match"]
        + output_df.at[team, "Average Zone Fouls per Match"]
        + output_df.at[team, "Average Stage Fouls per Match"]
        + output_df.at[team, "Average Over Exc Chute Fouls per Match"]])

### Percent of Different Defense Types in Matches AND Main Defense Type

In [77]:
output_df["Percent of Matches with No Defense"] = None
output_df["Percent of Matches with Some Defense"] = None
output_df["Percent of Matches with Full Defense"] = None
output_df["Main Defense Type"] = None

for team in team_list:
    
    defense_type_list = superapps_df.loc[superapps_df["metadata.robotTeam"] == team, "defense"].tolist()
    
    no_defense_count = 0
    some_defense_count = 0
    full_defense_count = 0
    
    for defense_type in defense_type_list:
        if defense_type == "noDef":
            no_defense_count += 1
        elif defense_type == "someDef":
            some_defense_count += 1
        elif defense_type == "fullDef":
            full_defense_count += 1

    defense_total_count = sum([no_defense_count, some_defense_count, full_defense_count])
        
    output_df.at[team, "Percent of Matches with No Defense"] = no_defense_count / defense_total_count
    output_df.at[team, "Percent of Matches with Some Defense"] = some_defense_count / defense_total_count
    output_df.at[team, "Percent of Matches with Full Defense"] = full_defense_count / defense_total_count
    
    biggest_defense_count = max([no_defense_count, some_defense_count, full_defense_count])
    main_defense_type = ""

    if biggest_defense_count == no_defense_count:
        main_defense_type += "No Defense "
    if biggest_defense_count == some_defense_count:
        main_defense_type += "Some Defense "
    if biggest_defense_count == full_defense_count:
        main_defense_type += "Full Defense"
    
    output_df.at[team, "Main Defense Type"] = main_defense_type

# FINALIZE AND SEND DATA

## Send Data

In [78]:
output_df.to_json("../server/static/analyzed_data.json", orient='records', default_handler=str)
output_df

,Auto Speaker Notes Near Avg,Auto Speaker Notes Mid Avg,Auto Speaker Notes Far Avg,Auto Speaker Notes Total Avg,Auto Notes Amp Avg,Auto Percent of Missed Notes,Auto Notes Avg,Tele Speaker Notes Near Avg,Tele Speaker Notes Mid Avg,Tele Speaker Notes Far Avg,...,Average Pin Bot Fouls per Match,Average Podium Fouls per Match,Average Zone Fouls per Match,Average Stage Fouls per Match,Average Over Exc Chute Fouls per Match,Average Fouls per Match,Percent of Matches with No Defense,Percent of Matches with Some Defense,Percent of Matches with Full Defense,Main Defense Type
metadata.robotTeam,,,,,,,,,,,,,,,,,,,,,
0,2.318182,1.909091,2.045455,6.272727,2.000000,1.039216,2.068182,5.227273,4.318182,3.318182,...,0.500000,0.409091,0.454545,0.454545,0.681818,4.363636,0.272727,0.363636,0.363636,Some Defense Full Defense
66,1.925926,1.962963,2.444444,6.333333,2.259259,1.096154,2.148148,4.444444,4.888889,3.666667,...,0.444444,0.592593,0.666667,0.481481,0.407407,4.555556,0.37037,0.185185,0.444444,Full Defense
143,1.866667,1.766667,1.700000,5.333333,1.866667,1.160714,1.800000,4.800000,5.133333,4.933333,...,0.533333,0.466667,0.666667,0.466667,0.400000,4.066667,0.233333,0.266667,0.5,Full Defense
429,1.740741,1.740741,2.185185,5.666667,2.592593,1.148936,2.064815,5.296296,5.148148,4.407407,...,0.444444,0.592593,0.629630,0.407407,0.481481,3.777778,0.444444,0.222222,0.333333,No Defense
433,1.880000,1.680000,2.040000,5.6,2.160000,1.021277,1.940000,4.920000,4.600000,4.960000,...,0.480000,0.560000,0.480000,0.440000,0.520000,3.96,0.32,0.44,0.24,Some Defense
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9555,1.777778,2.166667,2.166667,6.111111,2.111111,1.1875,2.055556,4.722222,3.777778,4.277778,...,0.555556,0.277778,0.500000,0.555556,0.388889,3.888889,0.333333,0.5,0.166667,Some Defense
9730,2.103448,2.482759,2.000000,6.586207,2.068966,1.196721,2.163793,4.655172,3.517241,4.448276,...,0.586207,0.620690,0.551724,0.482759,0.448276,4.310345,0.482759,0.172414,0.344828,No Defense
9771,2.150000,2.200000,1.700000,6.05,2.050000,0.906977,2.025000,4.200000,4.700000,4.100000,...,0.500000,0.450000,0.600000,0.400000,0.400000,3.45,0.3,0.3,0.4,Full Defense
